In [1]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import xgboost as xgb
#from catboost import CatBoostRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from scipy import sparse
import warnings
import time
import sys
import os
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
pd.set_option('max_colwidth',100)

In [2]:
train = pd.read_csv('jinnan_round1_train_20181227.csv', encoding = 'gb18030')
test  = pd.read_csv('jinnan_round1_testA_20181227.csv', encoding = 'gb18030')

In [3]:
train.head()

,样本id,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,收率
0,sample_1528,300,NaN,405.0,700,13:30:00,38.0,NaN,NaN,15:30:00,100,16:30:00,102,0.2,17:30:00,103.0,18:30:00,104.0,0.2,300,21:00-21:30,50.0,9.0,5.0,22:00:00,75,22:30:00,70,6:30-7:00,350.0,3.5,3.5,7:00-8:00,8:00:00,65,11:30:00,45.0,11:30-13:00,14:00-15:30,NaN,800.0,0.15,400,0.879
1,sample_1698,300,NaN,405.0,700,14:00:00,29.0,NaN,NaN,16:00:00,101,17:00:00,103,0.2,18:00:00,104.0,19:00:00,105.0,0.2,200,19:00-20:00,50.0,9.0,5.0,20:00:00,80,21:00:00,73,21:00-22:00,320.0,3.5,3.5,22:00-23:00,23:00:00,80,6:00:00,45.0,6:00-7:30,7:30-9:00,9:00-10:00,1200.0,0.15,400,0.902
2,sample_639,300,NaN,405.0,700,14:00:00,29.0,NaN,NaN,16:00:00,102,17:00:00,103,0.2,18:00:00,104.0,19:00:00,105.0,0.2,200,19:00-19:30,50.0,9.0,5.0,20:00:00,79,21:00:00,73,21:00-22:00,320.0,3.5,3.5,22:00-23:00,23:00:00,80,1:00:00,45.0,1:00-2:30,2:30-4:00,4:00-5:00,1200.0,0.15,400,0.936
3,sample_483,300,NaN,405.0,700,1:30:00,38.0,NaN,NaN,3:00:00,100,4:00:00,102,0.2,5:00:00,103.0,6:00:00,104.0,0.2,200,6:30-7:00,50.0,10.0,5.0,7:30:00,70,8:00:00,78,13:30-14:30,290.0,3.5,3.5,14:30-15:30,15:30:00,65,18:00:00,45.0,19:00-20:30,21:30-23:00,NaN,800.0,0.15,400,0.902
4,sample_617,300,NaN,405.0,700,22:00:00,29.0,NaN,NaN,0:00:00,101,1:00:00,103,0.2,2:00:00,104.0,3:00:00,105.0,0.2,200,3:00-4:00,50.0,9.0,5.0,4:00:00,80,5:00:00,73,5:00-6:00,320.0,3.5,3.5,6:00-7:00,7:00:00,80,9:00:00,45.0,9:00-10:30,10:30-12:00,12:00-13:00,1200.0,0.15,420,0.983


In [4]:
for df in [train, test]:
    df.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)

In [5]:
good_cols = list(train.columns)
# for col in train.columns:
#     rate = train[col].value_counts(normalize=True, dropna=False).values[0]
#     if rate > 0.9:
#         good_cols.remove(col)
#         print(col,rate)

train = train[train['收率']>0.865]
        
train = train[good_cols]
good_cols.remove('收率')
test  = test[good_cols]

In [6]:
target = train['收率']
del train['收率']
data = pd.concat([train,test],axis=0,ignore_index=True)
data = data.fillna(-1)

In [7]:
def timeTranSecond(t):
    try:
        t,m,s=t.split(":")
    except:
        if t=='1900/1/9 7:00':
            return 7*3600/3600
        elif t=='1900/1/1 2:30':
            return (2*3600+30*60)/3600
        elif t==-1:
            return -1
        else:
            return 0
    
    try:
        tm = (int(t)*3600+int(m)*60+int(s))/3600
    except:
        return (30*60)/3600
    
    return tm
for f in ['A5','A7','A9','A11','A14','A16','A24','A26','B5','B7']:
    data[f] = data[f].apply(timeTranSecond)

def getDuration(se):
    try:
        sh,sm,eh,em=re.findall(r"\d+\.?\d*",se)
    except:
        if se == -1:
            return -1 
        
    try:
        if int(sh)>int(eh):
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600 + 24
        else:
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600
    except:
        if se=='19:-20:05':
            return 1
        elif se=='15:00-1600':
            return 1
    
    return tm

def get_start(se):
    try:
        sh, sm, eh, em = re.findall(r'\d+\.?d*', se)
    except:
        if se == -1:
            return -1
    try:
        tm = (int(eh) * 3600 + int(em) * 60)/3600
    except:
        if se == '19:-20:05':
            return 19
        elif se == '15:00-1600':
            return 15
    return tm
for f in ['A20','A28','B4','B9','B10','B11']:
    data[f+'_diff'] = data.apply(lambda df: getDuration(df[f]), axis=1)
for f in ['A20','A28','B4','B9','B10','B11']:
    data[f] = data.apply(lambda df: get_start(df[f]), axis=1)

In [8]:
data['样本id'] = data['样本id'].apply(lambda x: int(x.split('_')[1]))

categorical_columns = [f for f in data.columns if f not in ['样本id']]
numerical_columns = [f for f in data.columns if f not in categorical_columns]

In [9]:
for f in categorical_columns:
    data[f] = data[f].map(dict(zip(data[f].unique(), range(0, data[f].nunique()))))
train = data[:train.shape[0]]
test  = data[train.shape[0]:]
print(train.shape)
print(test.shape)

(1389, 44)
(150, 44)


In [10]:
train['target'] = target
train['intTarget'] = pd.cut(train['target'], 5, labels=False)
train = pd.get_dummies(train, columns=['intTarget'])
li = ['intTarget_0.0','intTarget_1.0','intTarget_2.0','intTarget_3.0','intTarget_4.0']
mean_columns = []
for f1 in categorical_columns:
    cate_rate = train[f1].value_counts(normalize=True, dropna=False).values[0]
    if cate_rate < 0.90:
        for f2 in li:
            col_name = 'B14_to_'+f1+"_"+f2+'_mean'
            mean_columns.append(col_name)
            order_label = train.groupby([f1])[f2].mean()
            train[col_name] = train['B14'].map(order_label)
            miss_rate = train[col_name].isnull().sum() * 100 / train[col_name].shape[0]
            if miss_rate > 0:
                train = train.drop([col_name], axis=1)
                mean_columns.remove(col_name)
            else:
                test[col_name] = test['B14'].map(order_label)
                
train.drop(li+['target'], axis=1, inplace=True)
print(train.shape)
print(test.shape)

(1389, 149)
(150, 149)


In [11]:
X_train = train[mean_columns+numerical_columns].values
X_test = test[mean_columns+numerical_columns].values

In [12]:
# one hot
enc = OneHotEncoder()
for f in categorical_columns:
    enc.fit(data[f].values.reshape(-1, 1))
    X_train = sparse.hstack((X_train, enc.transform(train[f].values.reshape(-1, 1))), 'csr')
    X_test = sparse.hstack((X_test, enc.transform(test[f].values.reshape(-1, 1))), 'csr')
print(X_train.shape)
print(X_test.shape)

(1389, 1707)
(150, 1707)


In [13]:
y_train = target.values

In [14]:
# cat_params = {'learning_rate': 0.1,
#               'depth': 12,
#               'l2_leaf_reg': 10,
#               'bootstrap_type': 'Bernoulli',
#               'od_type': 'Iter',
#               'od_wait': 50,
#               'random_seed': 11,
#               'allow_writing_files': False}
# folds = KFold(n_splits=5, shuffle=True, random_state=666)
# oof_cat = np.zeros(len(train))
# predictions_cat = np.zeros(len(test))
# for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
#     print(f"fold {fold_+1}")
#     (trn_data_X, trn_data_y, val_data_X, val_data_y) = (
#         X_train.toarray()[trn_idx],
#         y_train[trn_idx],
#         X_train.toarray()[val_idx],
#         y_train[val_idx])
#     clf = CatBoostRegressor(iterations=20000, eval_metric='RMSE', use_best_model=True, **cat_params)
#     clf.fit(trn_data_X, trn_data_y,
#             eval_set=(val_data_X, val_data_y),
#             logging_level='Verbose')
#     oof_cat[val_idx] = clf.predict(val_data_X)
    
#     predictions_cat += clf.predict(X_test.toarray()) / folds.n_splits
    
# print("CV score: {:<8.8f}".format(mean_squared_error(oof_cat, target)))

In [15]:
param = {'num_leaves': 120,
         'min_data_in_leaf': 12, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8,
         "bagging_seed": 6,
         "metric": 'mse',
         "lambda_l1": 0.01,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=666)
oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print(f"fold {fold_+1}")
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, target)))

fold 1
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000232813	valid_1's l2: 0.000250289
[400]	training's l2: 0.000122191	valid_1's l2: 0.000163091
[600]	training's l2: 9.07421e-05	valid_1's l2: 0.000146635
[800]	training's l2: 7.51469e-05	valid_1's l2: 0.00014281
[1000]	training's l2: 6.55165e-05	valid_1's l2: 0.000142481
Early stopping, best iteration is:
[903]	training's l2: 6.96935e-05	valid_1's l2: 0.000142248
fold 2
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.0002322	valid_1's l2: 0.000249208
[400]	training's l2: 0.000123578	valid_1's l2: 0.000142156
[600]	training's l2: 9.22198e-05	valid_1's l2: 0.000118555
[800]	training's l2: 7.65112e-05	valid_1's l2: 0.000111791
[1000]	training's l2: 6.63761e-05	valid_1's l2: 0.000109989
[1200]	training's l2: 5.9367e-05	valid_1's l2: 0.000109886
Early stopping, best iteration is:
[1135]	training's l2: 6.13921e-05	valid_1's l2: 0.000109702
fold 3
Training until v

In [16]:
xgb_params = {'eta': 0.005, 'max_depth': 12, 'subsample': 0.8, 'colsample_bytree': 0.8, 
          'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 4}

folds = KFold(n_splits=5, shuffle=True, random_state=666)
oof_xgb = np.zeros(len(train))
predictions_xgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
    val_data = xgb.DMatrix(X_train[val_idx], y_train[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=100, params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(X_test), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, target)))

fold n°1
[0]	train-rmse:0.422663	valid_data-rmse:0.423395
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:0.256849	valid_data-rmse:0.257782
[200]	train-rmse:0.156534	valid_data-rmse:0.157515
[300]	train-rmse:0.095886	valid_data-rmse:0.097041
[400]	train-rmse:0.059347	valid_data-rmse:0.060597
[500]	train-rmse:0.037352	valid_data-rmse:0.038684
[600]	train-rmse:0.024145	valid_data-rmse:0.025711
[700]	train-rmse:0.016277	valid_data-rmse:0.01834
[800]	train-rmse:0.011635	valid_data-rmse:0.014455
[900]	train-rmse:0.008904	valid_data-rmse:0.012592
[1000]	train-rmse:0.007352	valid_data-rmse:0.011761
[1100]	train-rmse:0.006403	valid_data-rmse:0.011405
[1200]	train-rmse:0.005811	valid_data-rmse:0.011253
[1300]	train-rmse:0.005405	valid_data-rmse:0.011203
[1400]	train-rmse:0.005057	valid_data-rmse:0.011193
[1500]	train-rmse:0.004802	valid_data-rmse:0.011201
Stopping. Best i

In [17]:
oof_h2o = pd.read_csv('oof_h2o.csv', header=None).values.reshape(-1)
predictions_h2o = pd.read_csv('predictions_h2o.csv', header=None).values.reshape(-1)
print("CV score: {:<8.8f}".format(mean_squared_error(oof_h2o, target)))

CV score: 0.00005554


In [20]:
train_stack = np.vstack([oof_lgb, oof_xgb, oof_h2o]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb, predictions_h2o]).transpose()

In [21]:
pd.DataFrame(train_stack).corr()

,0,1,2
0,1.000000,0.982517,0.978979
1,0.982517,1.000000,0.978376
2,0.978979,0.978376,1.000000


In [18]:
folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=4590)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,target)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values
    
    clf_3 = LinearRegression()
    clf_3.fit(trn_data, trn_y)
    print(clf_3.coef_)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 10
    
mean_squared_error(target.values, oof_stack)

fold 0
[-0.48566608 -0.53455871  1.97526539]
fold 1
[-0.48128957 -0.52969888  1.96108894]
fold 2
[-0.46049419 -0.50909938  1.92552766]
fold 3
[-0.49912347 -0.4673784   1.92846176]
fold 4
[-0.5758198  -0.55793113  2.09283898]
fold 5
[-0.47873969 -0.544317    1.97859045]
fold 6
[-0.48650093 -0.5029391   1.94208937]
fold 7
[-0.52944712 -0.51479517  2.00356837]
fold 8
[-0.48748625 -0.56396863  2.0043011 ]
fold 9
[-0.51149206 -0.46934334  1.94308696]


3.2025431844908444e-05

In [19]:
sub_df = pd.read_csv('jinnan_round1_submit_20181227.csv', header=None)
sub_df[1] = predictions
sub_df[1] = sub_df[1].apply(lambda x:round(x, 3))
sub_df.to_csv("submit.csv", index=False, header=None)